In [49]:
import os
import tensorflow as tf 
import numpy as np 
import pandas as pd 
from PIL import Image
from tensorflow.keras import layers,models



In [50]:
data = os.path.join('C:/Users/hp/Documents/Machine learning/Deep learning/Projects(N)/Generative Adversarial Network(didnt do as no GPU)/DCGAN/archive')

In [51]:
img_size = 64
batch_size = 200
buffer_size = 50000
import zipfile


In [52]:
def load_image(directory,Img_Size):
    images = []
    for subdir, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(subdir, file)
            try:
                img = Image.open(file_path).convert('RGB')
                img = img.resize((Img_Size, Img_Size))
                img_array = np.array(img).astype('float32') #This converts the image from PIL format into a NumPy array, with data type float32. This array will have dimensions (Img_Size, Img_Size, 3), where 3 corresponds to the RGB channels.
                img_array = (img_array - 127.5) / 127.5
                images.append(img_array)
            except Exception as e:
                pass
    return np.array(images)

images_data = load_image(data, img_size)


In [53]:
len(images_data)

14965

In [54]:
images_data[0].shape

(64, 64, 3)

In [55]:
dataset = tf.data.Dataset.from_tensor_slices(images_data).shuffle(buffer_size).batch(batch_size)


#GENERATOR

In [56]:
dim = 100
def generators():
    model = models.Sequential([
        layers.Dense(8*8*512,input_dim=dim,activation='relu'),
        layers.BatchNormalization(),
        layers.Reshape((8,8,512)),
        layers.Conv2DTranspose(filters=256,kernel_size=(4,4),strides=(2,2),padding='same',activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2DTranspose(filters=128,kernel_size=(4,4),strides=(2,2),padding='same',activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2DTranspose(filters=64,kernel_size=(4,4),strides=(2,2),padding='same',activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(3,kernel_size=(4,4),padding='same',activation='tanh')
    ])
    return model
generator = generators()


c:\Users\hp\Downloads\Anaconda installations\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [57]:
generator.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 32768)          │     3,309,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 32768)          │       131,072 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_5 (Reshape)             │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_14             │ (None, 16, 16, 256)    │     2,097,408 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_15             │ (None, 32, 32, 128)    │       524,416 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_16             │ (None, 64, 64, 64)     │       131,136 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 64, 64, 3)      │         3,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,198,467 (23.65 MB)

 Trainable params: 6,132,035 (23.39 MB)

 Non-trainable params: 66,432 (259.50 KB)

#Discriminator

In [58]:
shape = (64,64,3)
def dicriminators():
    model = models.Sequential([
        layers.Conv2D(64,4,strides=(2,2),activation='relu',padding='same',input_shape=shape),
        layers.BatchNormalization(),
        layers.Conv2D(128,4,activation='relu',padding='same',strides=(2,2)),
        layers.BatchNormalization(),
        layers.Conv2D(256,4,strides=(2,2),activation='relu',padding='same'),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(1,activation='sigmoid')
    ])
    return model
dicriminator = dicriminators()

c:\Users\hp\Downloads\Anaconda installations\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [59]:
dicriminator.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 32, 32, 64)     │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 128)    │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 8, 8, 256)      │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │        16,385 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 677,057 (2.58 MB)

 Trainable params: 676,161 (2.58 MB)

 Non-trainable params: 896 (3.50 KB)

In [60]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

In [61]:
def dicriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [62]:
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

In [63]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=(1e-4))
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=(1e-4))

In [78]:
EPOCHS = 1
noise_dim = 100
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [79]:
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = dicriminator(images, training=True)
        fake_output = dicriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = dicriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, dicriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, dicriminator.trainable_variables))

In [80]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions + 1) / 2.0

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.close()

In [81]:
def train(datasets, epochs):
    for epoch in range(epochs):
        print(f'Starting epoch {epoch+1}/{epochs}')
        for image_batch in datasets:
            train_step(image_batch)
        generate_and_save_images(generator, epoch + 1, seed)  # Use the generator model, not the function

In [ ]:
train(dataset, EPOCHS)